In [2]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [3]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [4]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [5]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

dpir2023


In [6]:
S3_BUCKET_NAME='dpir2023'

In [7]:
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['divisas/', 'divisas/Tipocambio.csv', 'ipc/', 'ipc/IPC.csv']

In [22]:
import io

In [36]:
file = s3.Bucket(S3_BUCKET_NAME).Object('ipc/IPC.csv').get()
data=io.BytesIO(file['Body'].read())
df_ipc=pd.read_csv(data,sep=';')
df_ipc

,Año,Mes,Alimentos y bebidas no alcohólicas,Bebidas alcohólicas y tabaco,Prendas de vestir y calzado,"Vivienda, agua, electricidad, gas","Muebles, artículos para el hogar",Salud,Transporte,Comunicaciones,Recreación y cultura,Educación,Restaurante y hoteles,Bienes y servicios diversos
0,2011,Abril,105.01,101.43,100.78,103.99,101.12,101.30,106.80,100.68,100.79,100.77,101.48,101.31
1,2011,Mayo,105.71,102.15,101.07,105.21,101.39,101.62,107.25,100.77,100.49,100.77,102.01,101.60
2,2011,Junio,107.76,103.59,101.42,104.26,101.60,101.79,106.68,100.84,100.90,100.77,102.63,101.73
3,2011,Julio,110.66,104.82,101.72,104.42,101.78,101.98,106.85,100.83,101.15,100.82,103.21,101.79
4,2011,Agosto,112.13,105.20,102.01,104.92,102.25,102.24,107.26,100.77,101.08,100.82,103.63,102.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2022,Octubre,260.36,126.03,117.19,125.71,129.60,132.46,153.46,111.68,122.98,106.30,137.64,129.74
139,2022,Noviembre,258.21,126.54,117.71,126.07,130.19,132.81,156.14,111.55,124.35,106.30,138.80,130.02
140,2022,Diciembre,258.66,126.82,118.20,126.41,130.48,133.35,155.47,111.57,126.43,106.30,139.20,130.69
141,2023,Enero,261.69,127.12,118.44,126.51,130.58,133.56,150.99,111.57,127.10,106.65,140.34,131.06


In [38]:
file = s3.Bucket(S3_BUCKET_NAME).Object('divisas/Tipocambio.csv').get()
data=io.BytesIO(file['Body'].read())
df_TdC=pd.read_csv(data,sep=';',encoding='latin1')
df_TdC

,Divisa,Tipo de cambio
0,Quetzal Guatemalteco,1.0000
1,Dólar Estadounidense,7.7964
2,Euro,8.5030
3,Peso Mexicano,0.4315
4,Peso Filipino,0.1400
5,Rupia India,0.0948
6,Bolívar,0.3191
7,Peso Argentino,0.0371


In [13]:
import io

for remoteFile in remoteFileList:
    try:
        file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
        data = file['Body'].read()
        df_valor_divisas = pd.read_csv(file,encoding='latin1')
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df_valor_divisas.head()

No es un archivo.
Invalid file path or buffer object type: <class 'dict'>
No es un archivo.
Invalid file path or buffer object type: <class 'dict'>
No es un archivo.
Invalid file path or buffer object type: <class 'dict'>
No es un archivo.
Invalid file path or buffer object type: <class 'dict'>


,ï»¿AÃ±o;Mes;Alimentos y bebidas no alcohÃ³licas;Bebidas alcohÃ³licas y tabaco;Prendas de vestir y calzado;Vivienda,agua,electricidad,gas;Muebles,artÃ­culos para el hogar;Salud;Transporte;Comunicaciones;RecreaciÃ³n y cultura;EducaciÃ³n;Restaurante y hoteles;Bienes y servicios diversos
0,2011;Abril;105.01;101.43;100.78;103.99;101.12;...,NaN,NaN,NaN,NaN
1,2011;Mayo;105.71;102.15;101.07;105.21;101.39;1...,NaN,NaN,NaN,NaN
2,2011;Junio;107.76;103.59;101.42;104.26;101.6;1...,NaN,NaN,NaN,NaN
3,2011;Julio;110.66;104.82;101.72;104.42;101.78;...,NaN,NaN,NaN,NaN
4,2011;Agosto;112.13;105.2;102.01;104.92;102.25;...,NaN,NaN,NaN,NaN
